In [ ]:
%load_ext autoreload
%autoreload 2
import os; print(os.getcwd())
import socket; print(socket.gethostname())

import sys
sys.path.append("../")

import proper 
import mdtraj as md 
import Bio.SeqIO


In [ ]:
def find_first_comment_line(file_handle):
    for line in file_handle:
        if line.startswith("#"):
            return line.strip()
    return None  # If no comment line is found

def fix_header_length(input_str : str, linker_length :int) -> str:
    # Split the string by tab
    if '\t' not in input_str:
        raise ValueError("Input string must be in the format '#<number>\\t<number>'")
        
    part1, part2 = input_str.strip().split('\t')
    
    number1 = int(part1.strip('#'))
    
    # Add the value
    number1 += linker_length
    
    # Return in original format
    return f"#{number1}\t{part2}"

In [ ]:
## permute the a3m
a3m_file = 'TEV_trunc_dimer.a3m'
out_file = 'TEV_118-20AA.a3m'
linker = 'GGAGGEGSGGGSGGKGSGGT'
position0 = 118-1-6 


with open(a3m_file, "r") as a3m_handle:
    with open(out_file, "w") as out_handle:
        header = find_first_comment_line(a3m_handle)
        if header is None:
            #If no header start reading from the beginning for biopython. Looks like a regular fasta.
            a3m_handle.seek(0)
            # TODO make new header or just throw exception
        new_header = fix_header_length(header, len(linker))
        out_handle.write(new_header+'\n')
        seqs = list(Bio.SeqIO.parse(a3m_handle, "fasta"))
        first_len = len(seqs[0].seq)
        for seq in seqs:
            #print(seq.description)
            # Trunc all sequences to length of original seq
            permut_seq = proper.permute_seq_with_linker(seq.seq, position0=position0, linker=linker)
            out_handle.write('>'+seq.description+'\n')
            out_handle.write(str(permut_seq)+'\n')




